In [68]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Conv2D , MaxPooling2D , Flatten , Dense , Reshape , Input, Conv2DTranspose
from keras.datasets import mnist
from keras.utils import to_categorical
from sklearn.decomposition import PCA
from keras.models import Model
import matplotlib.pyplot as plt


In [69]:

#load the mnist database

(X_train , y_train ) , (X_test , y_test) = mnist.load_data()

X_train = X_train.astype('float32')
X_test = X_test.astype('float32')

#regularize the data
X_train /= 255
X_test /= 255

#standardize
X_train = (X_train - X_train.mean())/X_train.std()
X_test = (X_test - X_test.mean())/X_test.std()

#one hot encode y_train and y_test
y_train = to_categorical(y_train , num_classes = 10)
y_test = to_categorical(y_test , num_classes = 10)

#add gaussian noise to the data (with mean = 0 and std = noise_factor)
noise_factor = 0.8
X_train_noisy = X_train + noise_factor*tf.random.normal(shape = X_train.shape)
X_test_noisy = X_test + noise_factor*tf.random.normal(shape = X_test.shape)

In [70]:
#function that returns a smaller version of the train(or test) set with n_per_class number of samples per class

def batch_XY(X , y , n_per_class) :
    labels = np.unique(y)           #find the unique labels
    X_batch , y_batch = [] , []     #create empty lists
    for label in labels :

        indices = np.where(y == label)[0]           #find where y == label
        selected_indices = np.random.choice(indices , n_per_class)  #pick n_per_class of the items found in the indices of the previous line randomly
        X_batch.extend(X[selected_indices])                         #append to the corresponding lists
        y_batch.extend(y[selected_indices])


    #convert lists to np arrays
    X_batch = np.array(X_batch)
    y_batch = np.array(y_batch)
    #return the lists
    return (X_batch , y_batch)


In [71]:
#simple autoencoder with Dense layers

class Autoencoder(Model) :
  def __init__(self , hidden_size , latent_size , input_size) :
    super(Autoencoder ,self).__init__()

    #encoder
    self.encoder = Sequential([
        Input(shape = input_size) ,       #input layer with shape the shape of the mnist images
        Flatten() ,                       #flatten layer to flatten the input to a vector of elements
        Dense(hidden_size, activation = "relu") ,         #dense layer with hidden_size number of neurons
        Dense(latent_size , activation = "tanh")          #latent space layer where we get the encoded representation of the image
    ])

    self.encoder.summary()            #summary of the encoder model

    #decoder
    self.decoder = Sequential([
        Input(shape = self.encoder.output.shape),                     #input layer with shape the shape of the output of the encoder
        Dense(hidden_size , activation = "relu"),                     #dense layer with hidden_size number of neurons
        Dense(input_size[0]*input_size[1] , activation = "relu"),     #output layer with 28*28 number of neurons
        Reshape(input_size)                                           #reshape layer to get back the image
    ])

    self.decoder.summary()          #summary of the decoder model

  def call(self , x) :
    encoded = self.encoder(x)
    decoded = self.decoder(encoded)
    return decoded

In [72]:
#simple multilayer perceptron model for the mnist dataset

#the constructor takes 2 input arguements , the input size and a list of tupples called layers.
#Every tupple in the list has a number which corresponds to the number of neurons for the layer
#and a string which specifies the activation function

#NOTE I  : the input to this neural network is a 28*28 image which is then flattened
#NOTE II : the layers are the hidden layers , since the input layer is defined by the input size and the Input and Flatten layers.

class MLP(Model):

  def __init__(self , input_size , layers) :
    super(MLP , self).__init__()

    self.model = Sequential()
    self.model.add(Input(shape = input_size))
    self.model.add(Flatten())
    for i in range(len(layers)) :
        self.model.add(Dense(layers[i][0] , activation = layers[i][1]))


  def call(self , x) :
    out = self.model(x)
    return out


In [73]:
#convolutional autoencoder (this is the upgraded version of the simple autoencoder)

class CAE(Model):
  def __init__(self):
    super(CAE, self).__init__()
    self.encoder = tf.keras.Sequential([
      Input(shape=(28, 28, 1)),
      Conv2D(16, (3, 3), activation='relu', padding='same', strides=2),
      Conv2D(8, (3, 3), activation='tanh', padding='same', strides=2)
      ])

    self.decoder = tf.keras.Sequential([
      Conv2DTranspose(8, kernel_size=3, strides=2, activation='relu', padding='same'),
      Conv2DTranspose(16, kernel_size=3, strides=2, activation='relu', padding='same'),
      Conv2D(1, kernel_size=(3, 3), activation='relu', padding='same')])

  def call(self, x):
    encoded = self.encoder(x)
    decoded = self.decoder(encoded)
    return decoded

In [74]:
#a convolutional neural network (upgraded version of the multilayer perceptron)


class CNN(Model):

  def __init__(self) :
    super(CNN , self).__init__()

    self.model = Sequential()
    self.model.add(Conv2D(16 , kernel_size = (3 , 3) , strides = (1 , 1) , input_shape = (28 , 28 , 1) , activation = "relu"))
    self.model.add(MaxPooling2D(pool_size = (2 , 2)))
    self.model.add(Conv2D(32 , kernel_size = (3 , 3) , strides = (1 , 1) , input_shape = (28 , 28 , 1) , activation = "relu"))
    self.model.add(MaxPooling2D(pool_size = (2 , 2)))
    self.model.add(Conv2D(16 , kernel_size = (3 , 3) , strides = (1 , 1) , input_shape = (28 , 28 , 1) , activation = "relu"))
    self.model.add(Flatten())
    self.model.add(Dense(10 , activation = "softmax"))

  def call(self , x) :
    out = self.model(x)
    return out


**Multilayer perceptron models and denoising**

In [ ]:
#train an mlp network for the mnist dataset

#define the arcitecture
input_size = X_train.shape[1:]
print(input_size)
hidden_layers = [(300 , "relu") , (10 , "softmax")]

#instantiate model class
mlp = MLP(input_size , hidden_layers)

In [77]:
#define loss and optimizer
loss_mlp = tf.keras.losses.CategoricalCrossentropy()
optimizer_mlp = tf.keras.optimizers.Adam()
#define number of epochs and batch size and early stopping callback to avoid overfitting
epochs = 25
batch_size = 128
early_stopping_callback = tf.keras.callbacks.EarlyStopping(monitor = "val_loss" , patience = 5)
#compile the model
mlp.compile(loss = loss_mlp , optimizer = optimizer_mlp , metrics = ["accuracy"])

In [ ]:
#train the mlp network to the original data
mlp.fit(X_train , y_train , epochs = epochs , batch_size = batch_size , validation_split = 0.1 , callbacks =[early_stopping_callback])

In [ ]:
#test it
mlp.evaluate(X_test , y_test)

In [ ]:
#train an autoencoder to remove noise from the noisy data we created

hidden_size = 200
latent_size = 50
#define loss and optimizer for the autoencoder
loss_autoencoder = tf.keras.losses.MeanSquaredError()
optimizer_autoencoder = tf.keras.optimizers.Adam()
#create the model
input_size = X_train.shape[1:]
autoencoder = Autoencoder(hidden_size , latent_size , input_size)
autoencoder.compile(optimizer = optimizer_autoencoder, loss = loss_autoencoder)

In [ ]:
#train the autoencoder with the noisy data
autoencoder.fit(X_train_noisy , X_train , epochs = 50 , batch_size = 128)

In [ ]:
#noisy images denoised
encoded_imgs = autoencoder.encoder(X_test_noisy)
decoded_imgs = autoencoder.decoder(encoded_imgs)


In [ ]:
#print some reconstucted images to test the autoencoder

n = 11
plt.figure(figsize=(20, 4))
for i in range(n):

    # display original + noise
    ax = plt.subplot(2, n, i + 1)
    plt.title("original")
    plt.imshow(tf.squeeze(X_test_noisy[i]))
    plt.gray()
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)

    # display reconstruction
    bx = plt.subplot(2, n, i + n + 1)
    plt.title("reconstructed")
    plt.imshow(tf.squeeze(decoded_imgs[i]))
    plt.gray()
    bx.get_xaxis().set_visible(False)
    bx.get_yaxis().set_visible(False)
plt.show()


In [ ]:
#test the mlp on the reconstructed denoised images
mlp.evaluate(decoded_imgs , y_test)

**Convolutional models and denoising**

In [90]:
#we do the exact same things we did with the simple autoencoder and mlp model but with the convolutional models we created

#instantiate cnn
cnn = CNN()
#define loss and optimizer
loss_cnn = tf.keras.losses.CategoricalCrossentropy()
optimizer_cnn = tf.keras.optimizers.Adam()

#define epochs and batch size
epochs = 20
batch_size = 128
cnn.compile(loss = loss_cnn , optimizer = optimizer_cnn , metrics = ["accuracy"])

In [ ]:
#train with the original data
cnn.fit(X_train , y_train , epochs = epochs , batch_size = batch_size , validation_split = 0.1)

In [ ]:
#test
cnn.evaluate(X_test , y_test)

In [93]:
#create a convolutional autoencoder

cae = CAE()

loss_cae = tf.keras.losses.MeanSquaredError()
optimizer_cae = tf.keras.optimizers.Adam()
epochs = 50
batch_size = 128
cae.compile(loss = loss_cae , optimizer = optimizer_cae)

In [ ]:
#train
cae.fit(X_train_noisy , X_train, epochs = epochs , batch_size = batch_size)

In [ ]:
#noisy images denoised
encoded_imgs = cae.encoder(X_test_noisy)
decoded_imgs = cae.decoder(encoded_imgs)

In [ ]:
#evaluate the cnn model using the decoded images as input
cnn.evaluate(decoded_imgs , y_test)

In [ ]:
#evaluate the mlp model using the decoded images as input
mlp.evaluate(decoded_imgs , y_test)

In [ ]:
#plot some of the images
n = 11
plt.figure(figsize=(20, 4))
for i in range(n):

    # display original + noise
    ax = plt.subplot(2, n, i + 1)
    plt.title("original")
    plt.imshow(tf.squeeze(X_test_noisy[i]))
    plt.gray()
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)

    # display reconstruction
    bx = plt.subplot(2, n, i + n + 1)
    plt.title("reconstructed")
    plt.imshow(tf.squeeze(decoded_imgs[i]))
    plt.gray()
    bx.get_xaxis().set_visible(False)
    bx.get_yaxis().set_visible(False)
plt.show()


**Reconstruction with Principle Component Analysis**

In [ ]:
#pca as a method of reconstruction of images (no denoising)

from sklearn.decomposition import PCA

X_train_flat = np.reshape(X_train , (60_000, 28*28))      #flatten the images
X_test_flat = np.reshape(X_test, (10_000 , 28*28))

#perform pca and keep the first n components
n = 300
pca = PCA(n)
pca.fit(X_train_flat)
#apply the transformation
X_train_flat = pca.transform(X_train_flat)
X_test_flat = pca.transform(X_test_flat)
#reconstruct the images with the inverse transform
reconstructed = pca.inverse_transform(X_test_flat)
print(f"Percentage of variance = {sum(pca.explained_variance_ratio_)}")

In [ ]:
#plot some of the images
n = 11
plt.figure(figsize=(20, 4))
for i in range(n):

    # display original + noise
    ax = plt.subplot(2, n, i + 1)
    plt.title("original")
    plt.imshow(tf.squeeze(X_test[i]))
    plt.gray()
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)

    # display reconstruction
    bx = plt.subplot(2, n, i + n + 1)
    plt.title("reconstructed")
    plt.imshow(reconstructed[i].reshape((28,28)))
    plt.gray()
    bx.get_xaxis().set_visible(False)
    bx.get_yaxis().set_visible(False)
plt.show()